In [1]:
# --- ÉTAPE 1 : IMPORTS ET CHARGEMENT ---

# --- 1.1 Chargement des packages Python nécessaires ---
# Pour la manipulation des données (tableaux, calculs)
import pandas as pd
import numpy as np

# Pour interagir avec le système de fichiers (trouver nos CSV)
import os
import glob

# Pour la modélisation et l'évaluation en Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Pour gérer le déséquilibre des classes
from imblearn.over_sampling import SMOTE

# Pour sauvegarder notre modèle final et les transformateurs
import joblib

# Utilitaire pour un affichage plus propre
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliothèques importées.")

# --- 1.2 Chargement du dataset CIC-IDS2017 ---
# On spécifie le chemin vers le dossier contenant les fichiers
dataset_path = 'C:/Users/MTechno/Downloads/MachineLearningCVE'
# On utilise glob pour créer une liste de tous les fichiers .csv dans ce dossier
csv_files = glob.glob(os.path.join(dataset_path, "*.csv"))

if not csv_files:
    print("❌ ERREUR : Aucun fichier CSV trouvé dans le dossier 'data/'. Veuillez vérifier le chemin.")
else:
    # On charge chaque fichier et on les assemble en un seul grand tableau (DataFrame)
    df = pd.concat((pd.read_csv(f) for f in csv_files), ignore_index=True)
    print(f"✅ Dataset chargé. Il contient {df.shape[0]} lignes et {df.shape[1]} colonnes.")

✅ Bibliothèques importées.
✅ Dataset chargé. Il contient 2830743 lignes et 79 colonnes.


In [2]:
# --- ÉTAPE 2 : EXPLORATION ET NETTOYAGE ---

print(f"Dimensions avant nettoyage : {df.shape}")

# 2.1 Nettoyage des noms de colonnes (supprime les espaces comme ' Label' -> 'Label')
df.columns = df.columns.str.strip()

# 2.2 Remplacement des valeurs infinies (résultant de divisions par zéro) par NaN (Not a Number)
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# 2.3 Suppression des lignes contenant des valeurs manquantes (NaN)
df.dropna(inplace=True)

# 2.4 Suppression des lignes entièrement dupliquées
df.drop_duplicates(inplace=True)

# 2.5 Détection et suppression automatique des colonnes redondantes
# Une colonne est redondante si elle a la même valeur pour toutes les lignes (variance nulle)
cols_to_drop = [col for col in df.columns if df[col].nunique() == 1]
if cols_to_drop:
    df.drop(columns=cols_to_drop, inplace=True)
    print(f"Colonnes redondantes supprimées : {cols_to_drop}")

print(f"\n✅ Nettoyage terminé. Dimensions finales : {df.shape}")

Dimensions avant nettoyage : (2830743, 79)
Colonnes redondantes supprimées : ['Bwd PSH Flags', 'Bwd URG Flags', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate']

✅ Nettoyage terminé. Dimensions finales : (2520798, 71)


In [3]:
# --- ÉTAPE 3 : PRÉTRAITEMENT DES LABELS ---

# 3.1 Dictionnaire pour regrouper les attaques similaires
attack_mapping = {
    'BENIGN': 'BENIGN', 'DDoS': 'DDoS', 'PortScan': 'PortScan', 'Bot': 'Bot',
    'DoS GoldenEye': 'DoS', 'DoS Hulk': 'DoS', 'DoS Slowhttptest': 'DoS', 'DoS slowloris': 'DoS',
    'FTP-Patator': 'Brute Force', 'SSH-Patator': 'Brute Force',
    'Web Attack � Brute Force': 'Web Attack', 'Web Attack � XSS': 'Web Attack', 'Web Attack � Sql Injection': 'Web Attack',
    'Infiltration': 'Infiltration/Exploit', 'Heartbleed': 'Infiltration/Exploit'
}

# 3.2 Application du regroupement
df['Label'] = df['Label'].map(attack_mapping)
print("Nouvelle distribution des classes après regroupement :")
print(df['Label'].value_counts())

# 3.3 Séparation des données en caractéristiques (X) et cible (y)
X = df.drop(columns=['Label'])
y_temp = df['Label']

# 3.4 Encodage de la cible en valeurs numériques (ex: BENIGN -> 0, Bot -> 1, etc.)
le = LabelEncoder()
y = le.fit_transform(y_temp)

print("\n✅ Labels regroupés et encodés.")

Nouvelle distribution des classes après regroupement :
Label
BENIGN                  2095057
DoS                      193745
DDoS                     128014
PortScan                  90694
Brute Force                9150
Web Attack                 2143
Bot                        1948
Infiltration/Exploit         47
Name: count, dtype: int64

✅ Labels regroupés et encodés.


In [4]:
# --- ÉTAPE 4 : SÉPARATION DES DONNÉES ---

# On divise le dataset : 70% pour l'entraînement, 30% pour le test.
# 'stratify=y' garantit que la proportion de chaque attaque est la même dans les deux ensembles.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print("Données divisées en ensembles d'entraînement et de test.")
print(f"Taille de l'entraînement : {X_train.shape[0]} échantillons.")
print(f"Taille du test : {X_test.shape[0]} échantillons.")

Données divisées en ensembles d'entraînement et de test.
Taille de l'entraînement : 1764558 échantillons.
Taille du test : 756240 échantillons.


In [5]:
# --- ÉTAPE 5 : PRÉTRAITEMENT AVANCÉ ET RÉDUCTION DE DIMENSIONNALITÉ ---

# 5.1 Normalisation des features
# On apprend la mise à l'échelle sur X_train et on l'applique aux deux ensembles.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("1. Données normalisées.")

# 5.2 Application de SMOTE (uniquement sur les données d'entraînement)
print("\nDistribution des classes d'entraînement AVANT SMOTE :")
print(pd.Series(y_train).value_counts())

smote = SMOTE(random_state=42, n_jobs=-1)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

print("\nDistribution des classes d'entraînement APRES SMOTE :")
print(pd.Series(y_train_smote).value_counts())
print("\n2. Rééchantillonnage SMOTE appliqué.")

# 5.3 Réduction de la dimensionnalité avec PCA
# On apprend la PCA sur les données d'entraînement (équilibrées par SMOTE) et on l'applique aux deux ensembles.
pca = PCA(n_components=0.95, random_state=42) # Conserver 95% de la variance
X_train_pca = pca.fit_transform(X_train_smote)
X_test_pca = pca.transform(X_test_scaled) # On utilise le PCA appris sur le train
print("\n3. Réduction de dimensionnalité avec PCA appliquée.")
print(f"Nombre de caractéristiques réduit à {X_train_pca.shape[1]}.")

1. Données normalisées.

Distribution des classes d'entraînement AVANT SMOTE :
0    1466539
4     135621
3      89610
6      63486
2       6405
7       1500
1       1364
5         33
Name: count, dtype: int64

Distribution des classes d'entraînement APRES SMOTE :
4    1466539
6    1466539
0    1466539
3    1466539
2    1466539
7    1466539
1    1466539
5    1466539
Name: count, dtype: int64

2. Rééchantillonnage SMOTE appliqué.

3. Réduction de dimensionnalité avec PCA appliquée.
Nombre de caractéristiques réduit à 1.


In [ ]:
# --- ÉTAPE 6 : MODÉLISATION ET ÉVALUATION ---

# 6.1 Entraînement du modèle
print("\nEntraînement du RandomForestClassifier...")
model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train_pca, y_train_smote)
print("Modèle entraîné.")

# 6.2 Évaluation du modèle sur l'ensemble de test
print("\n--- Évaluation du modèle sur les données de test ---")
y_pred = model.predict(X_test_pca)

# On décode les prédictions pour un rapport lisible
y_pred_decoded = le.inverse_transform(y_pred)
y_test_decoded = le.inverse_transform(y_test)

print(classification_report(y_test_decoded, y_pred_decoded, digits=4))
print("\n✅ Modélisation et Évaluation terminées.")


Entraînement du RandomForestClassifier...


In [ ]:
# --- ÉTAPE 7 : SAUVEGARDE FINALE DES ARTEFACTS ---

# S'assurer que la bibliothèque pour la sauvegarde est importée
import joblib

print("--- Début de la sauvegarde des artefacts du pipeline ---")

# 1. Définir les noms des fichiers de sauvegarde
encoder_filename = 'final_label_encoder.joblib'
scaler_filename = 'final_scaler.joblib'
pca_filename = 'final_pca.joblib'
model_filename = 'final_model.joblib'

# 2. Sauvegarder chaque objet dans son propre fichier
#    - L'encodeur de labels (pour convertir 0, 1, 2... en 'BENIGN', 'DDoS', etc.)
joblib.dump(le, encoder_filename)
print(f"✅ Label Encoder sauvegardé sous : '{encoder_filename}'")

#    - Le normalisateur (pour mettre à l'échelle les nouvelles données)
joblib.dump(scaler, scaler_filename)
print(f"✅ Scaler sauvegardé sous : '{scaler_filename}'")

#    - Le réducteur de dimension (pour appliquer la PCA aux nouvelles données)
joblib.dump(pca, pca_filename)
print(f"✅ Objet PCA sauvegardé sous : '{pca_filename}'")

#    - Le modèle RandomForest entraîné lui-même
joblib.dump(model, model_filename)
print(f"✅ Modèle sauvegardé sous : '{model_filename}'")

print("\n--- Sauvegarde terminée ---")
print("Vous avez maintenant les 4 fichiers nécessaires pour utiliser votre modèle dans un autre script.")